# 2019 NBA Regular Season Evaluation

## Import needed packages and config

In [1]:
import requests
import mysql.connector
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup 


## Connect to MySql  and create database. Commented out once DB is created

In [3]:
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
)
cursor = cnx.cursor(buffered=True)

## (re)Connect to database, with database defined, to create table

In [4]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

## Make API Call to grab 2019 NBA reg season data


In [6]:

# headers = {'api-nba-v1.p.rapidapi.com' '{}'.format(api_key)}
headers = config.nba_headers

In [10]:


nba_data = []

response = requests.get(config.nba_url, headers=headers)
data = response.json()
nba_data.append(data)

In [12]:
nba_data 

[{'message': 'Missing RapidAPI application key. Go to https://docs.rapidapi.com/docs/keys to learn how to get your API application key.'}]

In [ ]:
nba_data[]


In [ ]:
nba_data[1]['vTeam']['shortName']

In [ ]:
parsed_games_nba = []
for game in nba_data:
    parsed_game = {}
    parsed_game['game_id'] = game['gameId']
    parsed_game['month'] = game['startTimeUTC'][5:7]
    parsed_game['away_team'] = game['vTeam']['shortName']
    parsed_game['away_team_score'] = game['vTeam']['score']['points']
    parsed_game['home_team'] = game['hTeam']['shortName']
    parsed_game['home_team_score'] = game['hTeam']['score']['points']
    parsed_games_nba.append(parsed_game)

In [ ]:
parsed_games_nba

In [ ]:
parsed_games_nba[0]['away_team_score'] = int(parsed_games_nba[0]['away_team_score'])

In [ ]:
print(type(parsed_games_nba[0]['away_team_score']))

In [ ]:
for i in range(len(parsed_games_nba)):
    parsed_games_nba[i]['home_team_score'] = int(parsed_games_nba[i]['home_team_score'])
#     parsed_game['home_team_score'] = game['hTeam']['score']['points']
#     parsed_games_nba.append(parsed_game)

## Creating NHL Table

In [ ]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [ ]:
# cursor.execute("""
# CREATE DATABASE MLB
# """)

# cursor.close()
# cnx.close()

In [ ]:
cursor.execute("""
                CREATE TABLE nba (
                game_id varchar(11) NOT NULL primary key,month varchar(2) not null,away_team varchar(11) not null,away_team_score int(3) not null,home_team varchar(11) not null,home_team_score int(3) not null
                )
""")

## Inserting to table

In [ ]:
nba_games_to_insert = []

for game in parsed_games_nba:

    nba_game_to_insert = (
        game['game_id'], 
        game['month'], 
        game['away_team'],
        game['away_team_score'],
        game['home_team'], 
        game['home_team_score']
    )
    
    nba_games_to_insert.append(nba_game_to_insert)

In [ ]:
print(type(nba_games_to_insert))

In [ ]:
cursor.execute("""TRUNCATE TABLE nba""")

In [ ]:
insert_query = """
    INSERT INTO nba (
                    game_id,
                    month,
                    away_team,
                    away_team_score,
                    home_team,
                    home_team_score
        )
    VALUES (
            %s,
            %s,
            %s,
            %s,
            %s,
            %s
            )
"""
cursor.executemany(insert_query, nba_games_to_insert)
cnx.commit()

### Questions (Null Hypothesis)

Is there a "home field" advantage(in general , 2018 season)?
    
Will a home team preform better (by month)?

If home team cover more than away team ("good teams win, great teams cover")?


## Creating MLB DataFrame

In [ ]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [ ]:
cursor.execute(  """SELECT * FROM nba
                    order by game_id;""")
#assigning database (pointed in MySql) as a dataframe using pandas                         
df_nba = pd.DataFrame(cursor.fetchall())


#assigning the column names 
df_nba.columns=['game_id','month', 'away_team', 'away_team_score', 'home_team', 'home_team_score', 'away_team_moneyline', 'home_team_moneyline', 'spread']

In [ ]:
df_nba